# Symmetric Encryption

# Introduction: 
Is a type of ciphers where there is only a one secret key used to perform the encryption and the decryption.

Examples:


● One-time pad .

● Advanced Encryption Standard (AES) .

● Data Encryption Standard (DES) .

● Data Encryption Standard 2 (DES2) .


# 1. One-time pad (OTP) :

The one-time pad relies on a number of principles that aren't immediately obvious to everyone, two of the most basic are:

 1-Any character can be represented as an integer.
 

 2-XORing 2 values, and then XORing the result of the values with one of the original values produces the other. So for example:
 

        a XOR b = c
        c XOR b = a
        c XOR a = b
    or
    

        text XOR pad = ciphertext
        ciphertext XOR pad = text
        ciphertext XOR text = pad
        
        
Additionally to maintain secrecy there are a few things that need to be done:


Each pad must be used one time, or else the ciphertexts are susceptible to cribdragging (a technique where you can use only resulting ciphertexts to determine the original pad).

The numbers generated must be random as flaws in the pseudorandom generation can be exploited.
The attacker must not have any intuition (be able to guess) as to what's inside the files.


# 1.2. ASCII Conversion  :
One way to do a conversion from string/chars to integers is to use an ASCII table, which allows you to have a 1-1 mapping of characters to an (at most) 2 byte integer. For example let's convert the string 'Hello' using the ASCII table below. The 'H' corresponds to 72 (0x48 in hex and 01001000 in binary) using the chart so that would be our first integer. The whole sequence would then be (72, 101, 108, 108, 111).


# 1.3. Pad Generation
From there we would use a random number generator to generate a pad of random 2 byte integers that is the same length as the input text (in this case 5 characters) lets say they are (11, 212, 8, 224, 122) which corresponds to 
("VT", "Ô", "BS", "à", "z" ) . Notice that not all of these are printable characters (any with two letters) and so the resulting text file may end up looking weird.

In [81]:
from secrets import choice   # Used to produce reliably random hex values
from string import printable # A list of printable characters to validate against
from pwn import xor
import os

In [2]:

def generate_pad(length):
    pad = ""
    for index in range(length):
        # Choose a random character
        pad_letter =  choice(printable)
        pad += (pad_letter)
    return pad

In [55]:
def encrypt_OTP(text, pad):
    result = ""
    for text_character, pad_character in zip(text, pad):
        if text_character not in printable:
            raise ValueError(f"Text value: {text_character} is not in ascii table ")
        # calculate XOR and find char
        xored_value = ord(text_character) ^ ord(pad_character)
        print(str(ord(text_character)) +" "+ str(ord(pad_character))+" "  +str (xored_value))
        result += chr(xored_value)
    return result

In [50]:
def decrypt_OTP(cipher,pad):
    result = ""

    for pad_character, ciphertext_number in zip(pad, cipher):
        xored_value = ord(pad_character) ^ ord(ciphertext_number)
        result += chr(xored_value)
        
    return result

In [76]:
message = "i love garage"
pad =generate_pad(len(message))

encrypted = encrypt_OTP(message,pad)
decrypted = decrypt_OTP(encrypted,pad)

print("Original message: " + message)
print("Encrypted message: " + encrypted)
print("Decrypted message: " + decrypted)

105 115 26
32 104 72
108 98 14
111 59 84
118 54 64
101 98 7
32 83 115
103 84 51
97 67 34
114 40 90
97 53 84
103 88 63
101 122 31
Original message: i love garage
Encrypted message: HT@s3"ZT?
Decrypted message: i love garage


In [78]:
print(103 ^ 88)
print (chr(103^ 88))

63
?


In [90]:
plaintext =b'GarAgE'
pad = os.urandom(len(plaintext))
print ("key "+ str(pad))
ciphertext=xor(plaintext,pad)
#print ciphertext
print("the encrypted message is :" +str(ciphertext))
#decrypt
plaintext = xor (ciphertext,pad)
#print plaintext
print("the decrypted message is :" +str(plaintext))

key b'\x11\x7fT\tw\xa5'
the encrypted message is :b'V\x1e&H\x10\xe0'
the decrypted message is :b'GarAgE'


In [93]:
#message = b'GarAgE{TH3_be$t_$3rvEr_1n_th3_WoRld}'
#pad =b'\x11\x7fT\tw\xa5'
ciphertext= b'V\x1e&H\x10\xe0j+\x1c:(\xc7t[ VS\x96c\t\x11{(\x94\x7f  aD\xfaF\x10\x06e\x13\xd8'
Hint=b'GarAgE'
l_hint=len(Hint)
for i in range(0,len(ciphertext)):
    key=xor(Hint,ciphertext[i:i+l_hint])
    plaintext=xor(ciphertext ,key)
    if Hint in plaintext :
        print(plaintext)
        break

b'GarAgE{TH3_be$t_$3rvEr_1n_th3_WoRld}'


# 2.  Advanced Encryption Standard (AES) :

In [102]:
from Crypto.Cipher import AES
from Crypto.Hash import SHA256



def encrypt_AES(msg,hkey) :
    BLOCK_SIZE= 16
    pad= '{'
    padding = lambda s: s+ (BLOCK_SIZE - len(s) % BLOCK_SIZE ) *pad
    cipher = AES.new(hkey,AES.MODE_ECB)
    result = cipher.encrypt(padding(msg).encode('utf-8'))
    return result 
def decrypt_AES(msg,hkey) :
    BLOCK_SIZE= 16
    pad= '{'
    decipher = AES.new(hkey,AES.MODE_ECB)
    result = decipher.decrypt(msg).decode('utf-8')
    pad_index =result.find(pad)
    result = result[:pad_index]
    return result 

In [109]:


message = 'garage'
hash_obj=SHA256.new(message.encode('utf-8'))
hkey= hash_obj.digest()
print(str(hkey))


encrypted = encrypt_AES(message,hkey)
decrypted = decrypt_AES(encrypted,hkey)

print("Original message: " + message)
print("Encrypted message: " + str(encrypted))
print("Decrypted message: " + str(decrypted))

b'!\xb7G\xde"\xdb\xb7W\xfd\xe3\xb6\x99\xaf\xe0W\x84\x9dNkI\xb66)\xe4\xc4\x0eu\xddG\x9d\xd3u'
Original message: garage
Encrypted message: b'\x12\xa7\x16\xad\xfb\xf2\x8c\xa4\xb4Rwn6UH\xdf'
Decrypted message: garage
